<a href="https://colab.research.google.com/github/Manisharout/My_Projects/blob/main/Task_Scheduler_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Task Scheduler System
# Implements Shortest Job First (SJF - Non-preemptive) and Round Robin Scheduling

class Process:
    def __init__(self, pid, arrival_time, burst_time):
        self.pid = pid
        self.arrival_time = arrival_time
        self.burst_time = burst_time
        self.remaining_time = burst_time
        self.waiting_time = 0
        self.turnaround_time = 0


def print_results(processes):
    total_wt = total_tat = 0
    print("\n{:<10}{:<15}{:<15}{:<15}{:<15}".format("PID", "Arrival", "Burst", "Waiting", "Turnaround"))
    for p in processes:
        print("{:<10}{:<15}{:<15}{:<15}{:<15}".format(
            p.pid, p.arrival_time, p.burst_time, p.waiting_time, p.turnaround_time))
        total_wt += p.waiting_time
        total_tat += p.turnaround_time

    print("\nAverage Waiting Time: {:.2f}".format(total_wt / len(processes)))
    print("Average Turnaround Time: {:.2f}".format(total_tat / len(processes)))


def print_gantt(gantt):
    print("\nGantt Chart:")
    print(" | ".join(gantt))


# ---- Shortest Job First (Non-preemptive) ----
def shortest_job_first(processes):
    time = 0
    completed = 0
    n = len(processes)
    visited = [False] * n
    gantt = []

    print("\n--- Shortest Job First Scheduling ---")

    while completed < n:
        idx = -1
        min_burst = float('inf')

        for i in range(n):
            if processes[i].arrival_time <= time and not visited[i]:
                if processes[i].burst_time < min_burst:
                    min_burst = processes[i].burst_time
                    idx = i

        if idx == -1:
            time += 1
        else:
            p = processes[idx]
            p.waiting_time = time - p.arrival_time
            gantt.append(f"P{p.pid}({time}→{time + p.burst_time})")
            time += p.burst_time
            p.turnaround_time = time - p.arrival_time
            visited[idx] = True
            completed += 1

    print_results(processes)
    print_gantt(gantt)


# ---- Round Robin Scheduling ----
from collections import deque

def round_robin(processes, quantum):
    time = 0
    queue = deque()
    gantt = []
    arrived = sorted(processes, key=lambda p: p.arrival_time)
    n = len(arrived)
    i = 0

    print("\n--- Round Robin Scheduling ---")

    queue.append(arrived[0])
    i = 1

    while queue:
        p = queue.popleft()

        exec_time = min(quantum, p.remaining_time)
        gantt.append(f"P{p.pid}({time}→{time + exec_time})")

        time += exec_time
        p.remaining_time -= exec_time

        # Add new arrivals
        while i < n and arrived[i].arrival_time <= time:
            queue.append(arrived[i])
            i += 1

        # Re-add if remaining time exists
        if p.remaining_time > 0:
            queue.append(p)
        else:
            p.turnaround_time = time - p.arrival_time
            p.waiting_time = p.turnaround_time - p.burst_time

    print_results(processes)
    print_gantt(gantt)


# ---- Main Function ----
def main():
    print("=== Task Scheduler System ===")
    n = int(input("Enter number of processes: "))

    processes = []
    for i in range(n):
        print(f"\nProcess {i+1}:")
        arrival = int(input("Arrival Time: "))
        burst = int(input("Burst Time: "))
        processes.append(Process(i + 1, arrival, burst))

    print("\nChoose Scheduling Algorithm:")
    print("1. Shortest Job First (SJF)")
    print("2. Round Robin (RR)")
    choice = int(input("Enter choice: "))

    if choice == 1:
        shortest_job_first([Process(p.pid, p.arrival_time, p.burst_time) for p in processes])
    elif choice == 2:
        quantum = int(input("Enter Time Quantum: "))
        round_robin([Process(p.pid, p.arrival_time, p.burst_time) for p in processes], quantum)
    else:
        print("Invalid choice!")


if __name__ == "__main__":
    main()


=== Task Scheduler System ===
Enter number of processes: 4

Process 1:
Arrival Time: 2
Burst Time: 3

Process 2:
Arrival Time: 4
Burst Time: 5

Process 3:
Arrival Time: 6
Burst Time: 7

Process 4:
Arrival Time: 7
Burst Time: 8

Choose Scheduling Algorithm:
1. Shortest Job First (SJF)
2. Round Robin (RR)
Enter choice: 2
Enter Time Quantum: 3

--- Round Robin Scheduling ---

PID       Arrival        Burst          Waiting        Turnaround     
1         2              3              -2             1              
2         4              5              0              0              
3         6              7              0              0              
4         7              8              0              0              

Average Waiting Time: -0.50
Average Turnaround Time: 0.25

Gantt Chart:
P1(0→3)
